#ALS recommendation
####Data Source https://movielens.org
####columns: UserID::MovieID::Rating::Timestamp

In [65]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [120]:
#read data
movie_data = sc.textFile("hdfs://bi-hd01:9000/user/bryan/ratings.dat")

In [121]:
pipeData = movie_data.map(lambda x : x.split('::'))

In [131]:
#pipeData.count()

In [58]:
#parse data
#format Rating(user=1, product=1, rating=1.0...)

In [123]:
parsedData = pipeData.map(lambda x: Rating(x[0], x[1], x[2]))

In [132]:
#parsedData.count()

# ALS Recommendation

In [181]:
rank = 20
numIterations = 5
lambda_=0.01 

In [182]:
model = ALS.train(parsedData, rank, numIterations, lambda_)

In [139]:
model.predict(100,2)

2.470825259642387

In [183]:
trainData = parsedData.map(lambda x : [x[0], x[1]])

In [144]:
trainData.take(5)

[[u'196', u'242'],
 [u'186', u'302'],
 [u'22', u'377'],
 [u'244', u'51'],
 [u'166', u'346']]

In [184]:
predData = model.predictAll(trainData).map(lambda x: ((x[0], x[1]),x[2]))

In [150]:
predData.take(5)

[((561, 160), 3.6665476836376976),
 ((1, 160), 3.721961414294466),
 ((331, 160), 4.160870652450875),
 ((13, 160), 3.974902415863322),
 ((653, 160), 3.4837504656637144)]

In [185]:
ratesAndPred = parsedData.map(lambda x : ((int(x[0]), int(x[1])), float(x[2]))).join(predData)

In [177]:
ratesAndPred.take(5)

[((452, 48), (5.0, 3.6782073083838203)),
 ((565, 855), (5.0, 4.73608260124074)),
 ((339, 81), (5.0, 4.271814587245729)),
 ((224, 196), (4.0, 3.7848737920049067)),
 ((296, 292), (5.0, 4.113539184632337))]

In [ ]:
#Mean Squared Error

In [186]:
MSE = ratesAndPred.map(lambda x: (x[1][0]-x[1][1]) ** 2).mean()

In [187]:
print MSE

0.34270134271
